In [1]:
import os
import struct
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.utils import shuffle
from neuralnet import NeuralNetMLP

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from numpy.random import seed
seed(1)

encoder = preprocessing.OneHotEncoder()
la_encoder = preprocessing.LabelEncoder()
scaler = preprocessing.StandardScaler()

Using TensorFlow backend.


In [2]:
def load_data():    
    data = pd.read_csv('./Data/train_cleaned_distr-imp.csv')
    data = data.drop(['id', 'subvillage','region_code','lga','ward','district_code','scheme_name','amount_tsh',
                      'public_meeting', 'permit'], axis=1)
    data['scheme_management'] = data['scheme_management'].fillna("Others")
    encoded = pd.DataFrame(encoder.fit_transform(data[['funder', 'installer', 'basin', 'region',
                                                       'scheme_management', 'extraction_type', 'management',
                                                       'payment_type', 'water_quality', 'quantity', 'source',
                                                       'waterpoint_type']]).toarray(),
                           columns=encoder.get_feature_names(['funder', 'installer', 'basin', 'region',
                                                              'scheme_management', 'extraction_type', 'management',
                                                              'payment_type', 'water_quality', 'quantity', 'source',
                                                              'waterpoint_type']))

    data = data.drop(['funder', 'installer', 'basin', 'region', 'scheme_management', 'extraction_type', 'management',
                  'payment_type', 'water_quality', 'quantity', 'source', 'waterpoint_type',
                     'longitude', 'latitude', 'longitude_imp_rand_choice', 'latitude_imp_rand_choice'], axis = 1)

    cols = data.columns.values

    data[cols] = scaler.fit_transform(data[cols])
    X = encoded.join(data)

    y = pd.read_csv('./Data/training_set_labels.csv')
    y = y[['status_group']]
    y = y.apply(la_encoder.fit_transform).values.ravel()
    return X, y

In [3]:
X, y = load_data()

C:\Anaconda\envs\py36\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Anaconda\envs\py36\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [4]:
y = np_utils.to_categorical(y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [9]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(80, activation='relu', kernel_initializer='random_normal', input_dim=157))
#Output Layer
classifier.add(Dense(3, activation='softmax', kernel_initializer='uniform'))
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='categorical_crossentropy', metrics =['accuracy'])

In [10]:
X_tmp = X_train[0:10000]
y_tmp = y_train[0:10000]

In [11]:
classifier.fit(X_tmp,y_tmp, batch_size=32, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
10000/10000 [==============================] - 1s 71us/step - loss: 0.9049 - acc: 0.5413
Epoch 2/100
10000/10000 [==============================] - 0s 40us/step - loss: 0.8877 - acc: 0.5419
Epoch 3/100
10000/10000 [==============================] - 0s 40us/step - loss: 0.8851 - acc: 0.5413
Epoch 4/100
10000/10000 [==============================] - 0s 37us/step - loss: 0.8816 - acc: 0.5439
Epoch 5/100
10000/10000 [==============================] - 0s 37us/step - loss: 0.8776 - acc: 0.5431
Epoch 6/100
10000/10000 [==============================] - 0s 37us/step - loss: 0.8725 - acc: 0.5502
Epoch 7/100
10000/10000 [==============================] - 0s 42us/step - loss: 0.8672 - acc: 0.5591
Epoch 8/100
10000/10000 [==============================] - 0s 37us/step - loss: 0.8600 - acc: 0.5643
Epoch 9/100
10000/10000 [==============================] - 0s 37us/step - loss: 0.8531 - acc: 0.5740
Epoch 10/100
10000/10000 [=================

10000/10000 [==============================] - 0s 43us/step - loss: 0.6092 - acc: 0.7107
Epoch 79/100
10000/10000 [==============================] - 0s 41us/step - loss: 0.6075 - acc: 0.7121
Epoch 80/100
10000/10000 [==============================] - 0s 40us/step - loss: 0.6074 - acc: 0.7101
Epoch 81/100
10000/10000 [==============================] - 0s 38us/step - loss: 0.6020 - acc: 0.7123
Epoch 82/100
10000/10000 [==============================] - 0s 43us/step - loss: 0.6023 - acc: 0.7070
Epoch 83/100
10000/10000 [==============================] - 0s 39us/step - loss: 0.6020 - acc: 0.7117
Epoch 84/100
10000/10000 [==============================] - 0s 37us/step - loss: 0.5990 - acc: 0.7135
Epoch 85/100
10000/10000 [==============================] - 0s 38us/step - loss: 0.5972 - acc: 0.7103
Epoch 86/100
10000/10000 [==============================] - 0s 39us/step - loss: 0.5976 - acc: 0.7121
Epoch 87/100
10000/10000 [==============================] - 0s 38us/step - loss: 0.5950 - acc: 

In [16]:
score = classifier.evaluate(X_test, y_test,verbose=1)
print('Accuracy: ' + str(score[1]))

17820/17820 [==============================] - 0s 20us/step
Accuracy: 0.4659371492437241
